In [ ]:
!pip install lime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2
from skimage.segmentation import mark_boundaries
from lime import lime_image
import os

In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip = True, rescale = 1/255, rotation_range = 30, zoom_range = 0.1)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test/'

In [ ]:
train_data = train_datagen.flow_from_directory(train_dir,
                                              target_size = (128,128),
                                               batch_size = 32,
                                               class_mode = 'binary'
                                              )
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size = (128,128),
                                               batch_size = 32,
                                               class_mode = 'binary'
                                              )

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape = (128,128,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data, epochs = 10, validation_data = test_data)

In [ ]:
h5_save_path = '/kaggle/working/pneumonia_detection_model.h5'
model.save(h5_save_path)
print(f"Model also saved to: {h5_save_path}")

In [ ]:
plt.plot(history.history['accuracy'],label="train_acc")
plt.plot(history.history['val_accuracy'],label="val_acc")
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

plt.plot(history.history['loss'],label="train_acc")
plt.plot(history.history['val_loss'],label="val_acc")
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
img_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg'
img = image.load_img(img_path, target_size = (128,128))
image_arr = image.img_to_array(img)/255
image_arr = np.expand_dims(image_arr, axis=0)
image_arr

In [ ]:
print("Model layers:")
for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}, {type(layer)}")

print("\nModel built status:", model.built)

img_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg'
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

print("\nTrying prediction...")
prediction = model.predict(img_array, verbose=0)
print("Prediction shape:", prediction.shape)

In [ ]:
def model_predict(images):

    preds = model.predict(images, verbose=0)

    result = np.zeros((images.shape[0], 2))
    result[:, 0] = 1 - preds.flatten()  # Normal probability
    result[:, 1] = preds.flatten()      # Pneumonia probability
    
    return result

In [ ]:
def explain_prediction_with_lime(image_path, model, top_labels=2, num_features=100000, num_samples=1000):

    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    x = img_array / 255.0  # Normalize
    
    pred = model.predict(np.expand_dims(x, axis=0), verbose=0)[0][0]
    
    img_filename = os.path.basename(image_path)
    
    explainer = lime_image.LimeImageExplainer()
    
    print("Generating LIME explanation... This may take a moment...")
    explanation = explainer.explain_instance(
        x, 
        model_predict, 
        top_labels=top_labels, 
        hide_color=0, 
        num_features=num_features,
        num_samples=num_samples
    )
    
    class_names = ['NORMAL', 'PNEUMONIA']
    
    pred_class_idx = 1 if pred > 0.5 else 0
    pred_class = class_names[pred_class_idx]
    
    probs = {
        'NORMAL': 1 - pred,
        'PNEUMONIA': pred
    }
    
    print(f"Prediction: {pred_class} with probability {probs[pred_class]:.4f}")
    
    temp, mask = explanation.get_image_and_mask(
        pred_class_idx, 
        positive_only=True, 
        num_features=5, 
        hide_rest=False
    )

    plt.figure(figsize=(15, 6))
    
    plt.suptitle(f"Image: {img_filename}\nPredicted: {pred_class} \nNORMAL: {probs['NORMAL']:.2%}, PNEUMONIA: {probs['PNEUMONIA']:.2%}",
             fontsize=12, y=1.05)  

    plt.subplots_adjust(top=0.85)  
    
    plt.subplot(131)
    plt.imshow(img)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(mark_boundaries(temp, mask))
    plt.title(f'LIME Explanation\nClass: {pred_class}')
    plt.axis('off')
    
    plt.subplot(133)

    ind = explanation.top_labels[0]
    dict_heatmap = dict(explanation.local_exp[ind])
    heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
    plt.imshow(heatmap, cmap='RdBu', vmin=-heatmap.max(), vmax=heatmap.max())
    plt.colorbar()
    plt.title('LIME Heatmap')
    plt.axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  
    plt.show()
    
    return explanation

In [ ]:
def generate_gradcam(model, img_array, last_conv_layer_name=None):

    predictions = model.predict(img_array, verbose=0)
    prob_pneumonia = predictions[0][0]
    prob_normal = 1 - prob_pneumonia

    if last_conv_layer_name is None:
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                last_conv_layer_name = layer.name
                print(f"Using convolutional layer: {last_conv_layer_name}")
                break
    
    last_conv_layer = model.get_layer(last_conv_layer_name)
    
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)
    
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer in model.layers[model.layers.index(last_conv_layer) + 1:]:
        x = layer(x)
    classifier_model = tf.keras.Model(classifier_input, x)
    
    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
       
        top_pred_index = 0 
        top_class_channel = preds[:, top_pred_index]
    
    
    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
    
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    
    return heatmap, {'NORMAL': prob_normal, 'PNEUMONIA': prob_pneumonia}

In [ ]:
def overlay_gradcam(image_path, heatmap, alpha=0.4, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    
    return superimposed_img

In [ ]:
def run_xai_analysis(image_path, method='both', gradcam_layer=None):

    if method.lower() not in ['lime', 'gradcam', 'both']:
        print("Method must be 'lime', 'gradcam', or 'both'")
        return
    
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array_norm = np.expand_dims(img_array, axis=0) / 255.0
    
    img_filename = os.path.basename(image_path)
    
    if method.lower() == 'lime' or method.lower() == 'both':
        lime_explanation = explain_prediction_with_lime(image_path, model)
    
    if method.lower() == 'gradcam' or method.lower() == 'both':
        heatmap, probs = generate_gradcam(model, img_array_norm, gradcam_layer)
        superimposed_img = overlay_gradcam(image_path, heatmap)
        
        pred_class = 'PNEUMONIA' if probs['PNEUMONIA'] > 0.5 else 'NORMAL'
        pred_prob = probs['PNEUMONIA'] if pred_class == 'PNEUMONIA' else probs['NORMAL']

        plt.figure(figsize=(15, 6))
        
        plt.suptitle(f"Image: {img_filename}\nPredicted: {pred_class} \nNORMAL: {probs['NORMAL']:.2%}, PNEUMONIA: {probs['PNEUMONIA']:.2%}", 
                     fontsize=12, y=0.98)
        
        plt.subplot(131)
        plt.imshow(cv2.cvtColor(cv2.resize(cv2.imread(image_path), (128, 128)), cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(132)
        plt.imshow(heatmap, cmap='jet')
        plt.title('GradCAM Heatmap')
        plt.axis('off')
        
        plt.subplot(133)
        plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
        plt.title('GradCAM Overlay')
        plt.axis('off')
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.85)  # Adjust to make room for suptitle
        plt.show()

In [ ]:
img_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg'

In [ ]:
run_xai_analysis(img_path, method='both')